In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import hockey_rink
import hockey_scraper

Gonna get familiar with hockey_scraper, never used it until now. Scraping the schedule between 2023-09-01 and 2024-08-31. Should be 1,312 NHL games in a season.

In [ ]:
hockey_scraper.scrape_seasons([2023,2024], True)

Noe have shift data for the nhl 2023-24 season as well as the play by play data (pbp) in separate csv files. 

In [ ]:
#Loading in pbp data
pbp=pd.read_csv("nhl_pbp_20232024.csv")

In [13]:
#Loading in shift data
shifts=pd.read_csv("nhl_shifts_20232024.csv")

In [15]:
shifts.head(5)

,Unnamed: 0,Game_Id,Period,Team,Player,Player_Id,Start,End,Duration,Date
0,0,20001,1,NSH,RYAN MCDONAGH,8474151.0,0.0,30.0,30.0,2023-10-10
1,1,20001,1,NSH,RYAN O'REILLY,8475158.0,0.0,35.0,35.0,2023-10-10
2,2,20001,1,T.B,VICTOR HEDMAN,8475167.0,0.0,41.0,41.0,2023-10-10
3,3,20001,1,NSH,FILIP FORSBERG,8476887.0,0.0,32.0,32.0,2023-10-10
4,4,20001,1,T.B,TYLER MOTTE,8477353.0,0.0,32.0,32.0,2023-10-10


In [12]:
pbp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445637 entries, 0 to 445636
Data columns (total 57 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       445637 non-null  int64  
 1   Game_Id          445637 non-null  int64  
 2   Date             445637 non-null  object 
 3   Period           445637 non-null  int64  
 4   Event            445637 non-null  object 
 5   Description      445637 non-null  object 
 6   Time_Elapsed     445637 non-null  object 
 7   Seconds_Elapsed  445637 non-null  float64
 8   Strength         445637 non-null  object 
 9   Ev_Zone          369193 non-null  object 
 10  Type             180765 non-null  object 
 11  Ev_Team          369085 non-null  object 
 12  Home_Zone        369193 non-null  object 
 13  Away_Team        445637 non-null  object 
 14  Home_Team        445637 non-null  object 
 15  p1_name          369011 non-null  object 
 16  p1_ID            360845 non-null  floa

Definitely do not need many of these variables so going to drop a bunch of them

In [18]:
pbp=pbp.drop(['Away_Coach','Home_Coach','Home_Goalie_Id','Home_Goalie','Away_Goalie_Id','Away_Goalie'], axis=1)

In [20]:
pbp.drop(['Unnamed: 0'], axis=1, inplace=True)

In [23]:
pbp[['awayPlayer1','awayPlayer2', 'awayPlayer3','awayPlayer4','awayPlayer5','awayPlayer6']].head(10)

,awayPlayer1,awayPlayer2,awayPlayer3,awayPlayer4,awayPlayer5,awayPlayer6
0,JUUSO PARSSINEN,RYAN O'REILLY,FILIP FORSBERG,RYAN MCDONAGH,ALEXANDRE CARRIER,JUUSE SAROS
1,JUUSO PARSSINEN,RYAN O'REILLY,FILIP FORSBERG,RYAN MCDONAGH,ALEXANDRE CARRIER,JUUSE SAROS
2,YAKOV TRENIN,RYAN O'REILLY,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
3,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
4,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
5,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
6,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
7,COLTON SISSONS,YAKOV TRENIN,COLE SMITH,LUKE SCHENN,ROMAN JOSI,JUUSE SAROS
8,CODY GLASS,GUSTAV NYQUIST,KIEFER SHERWOOD,JEREMY LAUZON,TYSON BARRIE,JUUSE SAROS
9,CODY GLASS,GUSTAV NYQUIST,KIEFER SHERWOOD,JEREMY LAUZON,TYSON BARRIE,JUUSE SAROS


Alright looks like players 1,2, and 3 are the forwards. 4 and 5 are the defensemen and 6 is the goalie, So i only need players 1-3

In [25]:
pbp=pbp.drop(['awayPlayer4','awayPlayer4_id','awayPlayer5','awayPlayer5_id','awayPlayer6','awayPlayer6_id','homePlayer4','homePlayer4_id','homePlayer5','homePlayer5_id','homePlayer6','homePlayer6_id'], axis=1)

Got rid of a ton of columns not needed. All columns now should be useful

In [31]:
pbp.columns

Index(['Game_Id', 'Date', 'Period', 'Event', 'Description', 'Time_Elapsed',
       'Seconds_Elapsed', 'Strength', 'Ev_Zone', 'Type', 'Ev_Team',
       'Home_Zone', 'Away_Team', 'Home_Team', 'p1_name', 'p1_ID', 'p2_name',
       'p2_ID', 'p3_name', 'p3_ID', 'awayPlayer1', 'awayPlayer1_id',
       'awayPlayer2', 'awayPlayer2_id', 'awayPlayer3', 'awayPlayer3_id',
       'homePlayer1', 'homePlayer1_id', 'homePlayer2', 'homePlayer2_id',
       'homePlayer3', 'homePlayer3_id', 'Away_Players', 'Home_Players',
       'Away_Score', 'Home_Score', 'xC', 'yC'],
      dtype='object')

Alright, now that we got rid of a good number of columns we don't need, let's think about the problem at hand. I want to evaluate who are the best 20 forward lines in the nhl. How should I accomplish this? I think the best way from a hollistic standpoint is expected goals %. Expected Goals % is the percent of all expected goals the player's team gets while the player is on the ice compared to the other team. For example, if a player's team gets 0.6 expected goals and the opposing team gets 0.4 expected goals while the player is on the ice, the player's expected goals % is 60%. 

On MoneyPuck, an xGoals% of above 60% gets a forward line inside the top 10 (Limiting to lines who played 200 minutes or more together).An xGoals% of 50% and above gets a FL inside the top 60 lines. 

I also want to look at each lines expected goals against and their expected goals for, rather than the percentage of expected goal they own. An expected goals for of 28 or above gets you in the top 10, and an xG For of 20 gets you inside the top 25. 

For xG against, 8.5 or less gets you inside the top 10, 11 or less gets you inside the top 25. 

By looking at these xG for and xG against rather than xG%, I can get the best offensive forward lines and the best defensive lines.

Thus, at the very end of this project my goal is to have 3 different rankings:
1) The best 15 lines in hockey (using xG%)
2) The best 15 offensive lines in hockey (using xG For)
3) The best 15 defensive lines in hockey (using xG Against)